# Load the data

In [9]:
import pandas as pd
in_datafile = "in/ttj.csv"
df = pd.read_csv(in_datafile)
df.head()

,parton_t_lep_Px,parton_t_lep_Py,parton_t_lep_Pz,parton_t_lep_E,parton_t_had_Px,parton_t_had_Py,parton_t_had_Pz,parton_t_had_E,parton_W_had_Px,parton_W_had_Py,...,b_lep_Pz,b_lep_E,b_had_Px,b_had_Py,b_had_Pz,b_had_E,W_lep_Px,W_lep_Py,W_lep_Pz,W_lep_E
0,8945.401616,24636.989673,-217674.294110,278985.014576,8283.042475,72058.624950,-1.431059e+06,1.443269e+06,-5097.826232,113901.961619,...,98667.859894,130343.023438,182480.734593,-86288.646457,126696.390732,238950.625000,-96273.537082,95614.983370,567624.355242,589988.274388
1,-80118.597231,46734.359967,-196631.866458,280012.115606,91834.412337,-30066.566498,-6.368368e+05,6.664149e+05,59056.754405,47526.569137,...,-38662.132083,92072.093750,24610.596015,-24885.721805,-1290.778056,35248.691406,3843.260993,-36125.297978,104974.895963,137125.985111
2,-139068.409348,-5914.922283,-55932.889577,227050.154282,134884.134442,12118.102464,1.223401e+05,2.515827e+05,141515.876349,49417.990863,...,92647.314813,104799.687500,12433.568687,39636.170358,167653.218337,172784.843750,64398.679896,-51801.203111,-50745.918019,125975.005508
3,-11284.473842,33422.206289,45305.217967,182036.566037,-20706.489425,-39816.182901,1.498502e+05,2.330612e+05,-24395.980797,-60288.888919,...,131924.418297,270595.156250,-17549.408595,21488.410919,25310.706765,37834.363281,-29762.306190,-98231.765180,481637.824359,498973.213324
4,37818.001459,-4590.763785,7451.532982,175849.295943,-80455.810381,-21964.950795,-6.285217e+04,2.021731e+05,-116825.975467,-9179.879824,...,-212533.902475,216230.812500,-4474.206790,59807.853057,-20406.806245,63570.820312,77349.883959,-78408.573493,-174891.759801,221770.231981


In [15]:
fraction_train = 0.65 
max_size = len(df)
train_df = df[:int(max_size*fraction_train)]
test_df  = df[int(max_size*fraction_train):max_size]
print "Created train dataframe of size: ", len(train_df)
print "Created test dataframe of size:  ", len(test_df)

Created train dataframe of size:  722
Created test dataframe of size:   390


### Create input X and Y vectors

Set-up the input and output vectors. 

The output vector is going to be the jet indices:
    ( 
    
        is_fisrt_bjet_leptonic
        is_second_bjet_hadronic,
        is_in_W
        is_in_W,
        is_in_W,
    )
Then some spectator variables needed for the custom loss function that minimizes differences in four momenta: 
    (
    
        parton_top_had_p4,
        parton_top_lep_p4,
        parton_w_had_p4,
        reco_w_lep_p4,
        reco_bjet_1
        reco_bjet_2
        reco_jet_1
        reco_jet_2
        reco_jet_3     
    )
    

In [315]:
import numpy as np
"""
jet_1       
jet_2       
jet_3       
bjet_1      
bjet_2      
t_lep_truth 
t_had_truth 
w_had_truth
w_lep_truth 
"""
all_variables = list(df.columns.values)
reco_variables = all_variables[12:24]+all_variables[32:40]
# parton_variables = all_variables[0:12]+all_variables[24:32]+all_variables[40:44]
parton_variables = all_variables[0:12]+all_variables[40:44]

jet_input_train   =train_df[reco_variables].values
Y_train = truth_input_train = train_df[reco_variables+parton_variables].values
jet_input_test   =test_df[reco_variables].values
Y_test = truth_input_test = test_df[reco_variables+parton_variables].values

# Add the dummy assignment vectors 
# A
for i in xrange(3):
    Y_train =  np.insert(Y_train, len(Y_train[0]),0, axis=1)
    Y_test =  np.insert(Y_test, len(Y_test[0]),0, axis=1)



print jet_input_train.shape
print truth_input_train.shape
print Y_train.shape

(722, 20)
(722, 36)
(722, 39)


# Basic plots

# Set-up the model

In [312]:
import ROOT as r 
from keras import backend as K

def transform_to_jet_assignment(y_pred):
    """
        Convert a 3-vector that sums to unity into a vector of 3 predictions
        where:
            first element represents  if jet_1 is in the  hadronic W
            second element represents if jet_2 is in the  hadronic W
            third element represents  if bjet_1 is in the hadronic top
            
            The jet_3 is therefore in the hadronic_W when (2 - [0]-[1] ) == 1
            The bjet_2 is therefore in the hadronic_W when (1 - [2] ) == 1
    """
#     y_pred[:,K.argmax(y_pred[:,:2])]  = 1.0
#     y_pred[:,K.argmin(y_pred[:,1:3])] = 1.0
    y_pred[:,K.argmin(y_pred[:,:3])]  *= 0.0
    return y_pred

def mass_diff_loss(y_true,y_pred):
    """
        user defined evaluation function, return a pair (metric_name, result)        
        See the following links for details 
            https://github.com/keras-team/keras/issues/4781
            https://stackoverflow.com/questions/45961428/make-a-custom-loss-function-in-keras
        
        
        Intended to be used with 16 so-called spectator variables that correspond to
        the truth level leptonic and hadronic top, truth level hadronic W, and reconstucted
        leptonic W boson four momemtum
        
        A further 20 spectator variables are passed that correspond to the reconstruction level
        jets, 3 light jets and 2 b-jets. 
        
        The only part of y_pred that effects this loss function is the last 3 elements. These 
        correspond to jet assignment. 
    """
    # Note everything is done by tensor algebra - so this actually working things out for
    # every event in a batch all in one go using a highly optimized tensorflow backend
        
    # Transform predictions into booleans that represent
    # if a jet is a decay product of the hadronic top quark 
    assignment = y_pred[:,:3]#transform_to_jet_assignment(y_pred) 
#     assignment[:,K.argmax(assignment[:,:2])]
#     t = y_pred[:,K.argmax(assignment[:,:2])]
#     assignment[:,0].assign_slice( 1.0)
#     assignment[:,:].assign(K.zeros( 3 ))
#     assignment[:,0] = 0.0

    # Get the four vectors for each of the needed physics objects
    # we're in a basis such that each four vector = (px,py,pz,E)
    jet_1        = y_pred[:,0:4]/1e3
    jet_2        = y_pred[:,4:8]/1e3
    jet_3        = y_pred[:,8:12]/1e3
    bjet_1       = y_pred[:,12:16]/1e3
    bjet_2       = y_pred[:,16:20]/1e3
    t_lep_truth  = y_pred[:,20:24]/1e3
    t_had_truth  = y_pred[:,24:28]/1e3
    w_had_truth  = y_pred[:,28:32]/1e3
    w_lep_truth  = y_pred[:,32:36]/1e3

    # calculated the masses for all of the predictions and truth level objects
    top_hadronic_px  = assignment[:,0]*jet_1[:,0] + assignment[:,1]*jet_2[:,0] + (2-assignment[:,1])*jet_3[:,0] + assignment[:,2]*bjet_1[:,0]+ (1-assignment[:,2])*bjet_2[:,0]
    top_hadronic_py  = assignment[:,0]*jet_1[:,1] + assignment[:,1]*jet_2[:,1] + (2-assignment[:,1])*jet_3[:,1] + assignment[:,2]*bjet_1[:,1]+ (1-assignment[:,2])*bjet_2[:,1]
    top_hadronic_pz  = assignment[:,0]*jet_1[:,2] + assignment[:,1]*jet_2[:,2] + (2-assignment[:,1])*jet_3[:,2] + assignment[:,2]*bjet_1[:,2]+ (1-assignment[:,2])*bjet_2[:,2]
    top_hadronic_e   = assignment[:,0]*jet_1[:,3] + assignment[:,1]*jet_2[:,3] + (2-assignment[:,1])*jet_3[:,3] + assignment[:,2]*bjet_1[:,3]+ (1-assignment[:,2])*bjet_2[:,3]    
    top_hadronic_m   = K.sqrt(-K.square(top_hadronic_e)+K.square(top_hadronic_px)+K.square(top_hadronic_py)+K.square(top_hadronic_py))

    w_hadronic_px  = assignment[:,0]*jet_1[:,0] + assignment[:,1]*jet_2[:,0]  + (2-assignment[:,1])*jet_3[:,0]
    w_hadronic_py  = assignment[:,0]*jet_1[:,1] + assignment[:,1]*jet_2[:,1]  + (2-assignment[:,1])*jet_3[:,1]
    w_hadronic_pz  = assignment[:,0]*jet_1[:,2] + assignment[:,1]*jet_2[:,2]  + (2-assignment[:,1])*jet_3[:,2]
    w_hadronic_e   = assignment[:,0]*jet_1[:,3] + assignment[:,1]*jet_2[:,3]  + (2-assignment[:,1])*jet_3[:,3]
    w_hadronic_m   = K.sqrt(-K.square(w_hadronic_e) + K.square(w_hadronic_px)+K.square(w_hadronic_py)+K.square(w_hadronic_pz))
    
    t_lep_truth_m   = K.sqrt(-K.square(t_lep_truth[:,3]) + K.square(t_lep_truth[:,0]) + K.square(t_lep_truth[:,1]) + K.square(t_lep_truth[:,2]))
    t_had_truth_m   = K.sqrt(-K.square(t_had_truth[:,3]) + K.square(t_had_truth[:,0]) + K.square(t_had_truth[:,1]) + K.square(t_had_truth[:,2]))
    w_had_truth_m   = K.sqrt(-K.square(w_had_truth[:,3]) + K.square(w_had_truth[:,0]) + K.square(w_had_truth[:,1]) + K.square(w_had_truth[:,2]))
    w_lep_truth_m   = K.sqrt(-K.square(w_lep_truth[:,3]) + K.square(w_lep_truth[:,0]) + K.square(w_lep_truth[:,1]) + K.square(w_lep_truth[:,2]))

    top_lep_pz = w_lep_truth[:,0] + (1-assignment[:,2])*bjet_1[:,0]+ (assignment[:,2])*bjet_2[:,0]
    top_lep_py = w_lep_truth[:,1] + (1-assignment[:,2])*bjet_1[:,1]+ (assignment[:,2])*bjet_2[:,1]
    top_lep_px = w_lep_truth[:,2] + (1-assignment[:,2])*bjet_1[:,2]+ (assignment[:,2])*bjet_2[:,2]
    top_lep_e  = w_lep_truth[:,3] + (1-assignment[:,2])*bjet_1[:,3]+ (assignment[:,2])*bjet_2[:,3]
    top_lep_m  = -K.square(top_lep_e) + (K.square(top_lep_px)+K.square(top_lep_py)+K.square(top_lep_pz))
    # Return the loss as the differences in mass for each of the reconstructed objects 
    return K.square(top_lep_m-t_lep_truth_m) + K.square(top_hadronic_m-t_had_truth_m)+K.square(w_hadronic_m-w_had_truth_m)


In [313]:
from keras.layers import Input, Dense,Concatenate,concatenate
from keras.models import Model
from keras.utils import plot_model

# Construct the jet assignment part of the network, a simple MLP
jet_assignment_input  = Input(shape=(20,), dtype='float32', name='RecoJetInput')
jet_assignment_hidden = Dense(32, activation='relu', name='JetAssignmentLayer')(jet_assignment_input)
jet_assignment_output = Dense(3,  activation='softmax', name='JetAssignment2ndLayer')(jet_assignment_hidden)

# Potenttially might want to add additional W mass reconstruction part of the network here??
# auxiliary_output = Dense(3, activation='sigmoid', name='aux_output')(jet_assignment_input)

# Merge in truth level information required for the 
auxiliary_input = Input(shape=(36,), name='TruthLevelInput')
total_model = concatenate([ auxiliary_input,jet_assignment_output], name='Combination')

# And finally we add the main logistic regression layer
# output = Dense(3, activation='sigmoid', name='Output')(x)

model = Model(inputs=[jet_assignment_input,auxiliary_input], outputs=[total_model])
model.compile(optimizer='rmsprop', loss=mass_diff_loss)
model.summary()
plot_model(model, to_file='model.png',show_shapes=True)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
RecoJetInput (InputLayer)        (None, 20)            0                                            
____________________________________________________________________________________________________
JetAssignmentLayer (Dense)       (None, 32)            672         RecoJetInput[0][0]               
____________________________________________________________________________________________________
TruthLevelInput (InputLayer)     (None, 36)            0                                            
____________________________________________________________________________________________________
JetAssignment2ndLayer (Dense)    (None, 3)             99          JetAssignmentLayer[0][0]         
___________________________________________________________________________________________

# Train the model

In [314]:
model.fit( [jet_input_train,truth_input_train],
          Y_train, 
          validation_split = 0.15,
#           batch_size=50,
          epochs = 15
         )

Train on 613 samples, validate on 109 samples
Epoch 1/15
613/613 [==============================] - 7s - loss: 808941145562171544403157140373504.0000 - val_loss: 7146739755615783733801584164864.0000
Epoch 2/15
613/613 [==============================] - 0s - loss: 808941079479978712228308507951104.0000 - val_loss: 7146739755615783733801584164864.0000
Epoch 3/15
613/613 [==============================] - 0s - loss: 808941086130180118826576907337728.0000 - val_loss: 7146739755615783733801584164864.0000
Epoch 4/15
613/613 [==============================] - 0s - loss: 808941087965139144908586321182720.0000 - val_loss: 7146739755615783733801584164864.0000
Epoch 5/15
613/613 [==============================] - 0s - loss: 808941056321713111782195984334848.0000 - val_loss: 7146739755615783733801584164864.0000
Epoch 6/15
613/613 [==============================] - 0s - loss: 808941061957942248817706150133760.0000 - val_loss: 7146739755615783733801584164864.0000
Epoch 7/15
613/613 [================

In [327]:
test_pred = model.predict([jet_input_test,truth_input_test])


In [335]:
scores[3,-3:]


array([ 1.,  0.,  0.], dtype=float32)